In this notebook, we'll explore different forms of A/B testing

# Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
import numpy as np
from math import ceil

# Setup and Functions

In [4]:
sns.set_theme()

# Test 1: Conversion Rate

This test is based on a website which recently changed its webpage design. The previous webpage had a conversion rate of 13%, and the desired effect is an increase in conversions to 15%.

## Data Preparation

In [9]:
df1 = pd.read_csv("data/ab_data.csv")

In [10]:
df1.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [42]:
# We won't be using the time in this analysis, so we'll drop it.
df1 = df1.drop(columns=["timestamp"])

In [43]:
df1.groupby(["group", "landing_page"]).count()

user_id  converted
group     landing_page                    
control   new_page         1928       1928
          old_page       145274     145274
treatment new_page       145311     145311
          old_page         1965       1965

There are some users in the control group with the new page and vice-versa, we'll remove them.

In [44]:
df2 = df1.loc[((df1["group"] == "control") & (df1["landing_page"] == "old_page")) | ((df1["group"] == "treatment") & (df1["landing_page"] == "new_page")) ]

In [45]:
df2.groupby(["group", "landing_page"]).count()

,,user_id,converted
group,landing_page,,
control,old_page,145274,145274
treatment,new_page,145311,145311


## Effect Size and Power Calculation

We have over 145,000 datapoints for each webpage, that's plenty of data. Collecting this amount of data for an e-commerce website could take a long time and be expensive if it fails. For an hypothetical, we could calculate the minimum amount of data to collect in order to detect the desired effect of increased conversion. For that, we'll calculate the effect size and perform a power calculation to find the numbers.

In [47]:
eff_size = sms.proportion_effectsize(0.13, 0.15)
print(f"The effect size is {eff_size}")

The effect size is -0.0576728617308947


In [51]:
nobs = sms.NormalIndPower().solve_power(effect_size=eff_size, alpha=0.05, power=0.8)
nobs = ceil(nobs)
print(f"The number of observations needed to find the effect size is {nobs}")

The number of observations needed to find the effect size is 4720


Therefore, with 4720 observations, there's an 80% chance of observing the effect, if it exists.

## Hypothesis Testing

For the sake of the exercise, we'll use a sample of 4720 observations from each group, the minimum established in the last step.

In [99]:
control_df = df2[df2["group"] == "control"].sample(n=nobs, random_state=1)
treatment_df = df2[df2["group"] == "treatment"].sample(n=nobs, random_state=1)

df_test = pd.concat ([control_df, treatment_df], axis=0)

In [100]:
df_test.groupby("group")["converted"].mean()

group
control      0.118644
treatment    0.111441
Name: converted, dtype: float64

Uh oh. It looks like the treatment group has in fact a *lower* conversion rate than the control. That pretty much rules out the desired effect of increased sales, but we'll run the test anyway. Maybe it turns out the decrease effect is statistically significant.

In [101]:
count_list = [control_df["converted"].sum(), treatment_df["converted"].sum()]
nobs_list = [len(control_df), len(treatment_df)]

In [102]:
z_stat, p_val = sms.proportions_ztest(count=count_list, nobs=nobs_list)

In [103]:
print(f"The Z Statistic is {z_stat:.2f} and the p-value is {p_val:.3f}")

The Z Statistic is 1.10 and the p-value is 0.273


## Conclusion

According to our test, the result is negative: the new webpage design didn't improve the conversion rate of the website, although at least there isn't enough evidence that it hurt conversion rates either. The p-value found, much above the set target 0.05, shows evidence for the null hypothesis that the two webpage designs are not significantly different in terms of conversion.